<a href="https://colab.research.google.com/github/ngolemai020405/Thuchanh_TTNT/blob/main/BaoCao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import copy
from heapq import heappush, heappop

n = 4
rows = [1, 0, -1, 0]
cols = [0, -1, 0, 1]

class PriorityQueue:
    def __init__(self):
        self.heap = []

    def push(self, key):
        heappush(self.heap, key)

    def pop(self):
        return heappop(self.heap)

    def empty(self):
        return len(self.heap) == 0

class Node:
    def __init__(self, parent, mats, empty_tile_posi, costs, levels):
        self.parent = parent
        self.mats = mats
        self.empty_tile_posi = empty_tile_posi
        self.costs = costs
        self.levels = levels

    def __lt__(self, nxt):
        return self.costs < nxt.costs

def calculateCosts(mats, final) -> int:
    count = 0
    for i in range(n):
        for j in range(n):
            if mats[i][j] != 0 and mats[i][j] != final[i][j]:
                count += 1
    return count

def newNodes(mats, empty_tile_posi, new_empty_tile_posi, levels, parent, final) -> Node:
    new_mats = copy.deepcopy(mats)

    x1, y1 = empty_tile_posi
    x2, y2 = new_empty_tile_posi
    new_mats[x1][y1], new_mats[x2][y2] = new_mats[x2][y2], new_mats[x1][y1]

    costs = calculateCosts(new_mats, final)

    return Node(parent, new_mats, new_empty_tile_posi, costs, levels)

def printMats(mats):
    for row in mats:
        print(" ".join(str(num) for num in row))
    print()

def isSafe(x, y):
    return 0 <= x < n and 0 <= y < n

def printPath(root):
    if root is None:
        return
    printPath(root.parent)
    printMats(root.mats)

def solve(initial, empty_tile_posi, final):
    pq = PriorityQueue()
    costs = calculateCosts(initial, final)
    root = Node(None, initial, empty_tile_posi, costs, 0)
    pq.push(root)

    while not pq.empty():
        minimum = pq.pop()

        if minimum.costs == 0:
            print("Giải pháp tìm thấy:")
            printPath(minimum)
            return

        for i in range(4):
            new_tile_posi = [
                minimum.empty_tile_posi[0] + rows[i],
                minimum.empty_tile_posi[1] + cols[i],
            ]

            if isSafe(new_tile_posi[0], new_tile_posi[1]):
                child = newNodes(minimum.mats,
                                 minimum.empty_tile_posi,
                                 new_tile_posi,
                                 minimum.levels + 1,
                                 minimum, final)

                pq.push(child)

initial = [
    [1, 2, 3, 4],
    [5, 6, 0, 8],
    [9, 10, 7, 11],
    [13, 14, 15, 12]
]

final = [
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 15, 0]
]

empty_tile_posi = [1, 2]

solve(initial, empty_tile_posi, final)